In [ ]:
import findspark
findspark.init('/home/ductien/spark-3.3.2-bin-hadoop3')
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, col

In [2]:
#create Spark Session
spark = SparkSession.builder.appName('cleaning').getOrCreate()

23/04/28 01:34:45 WARN Utils: Your hostname, DT-Kubuntu resolves to a loopback address: 127.0.1.1; using 192.168.2.143 instead (on interface wlp3s0)
23/04/28 01:34:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/28 01:34:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/28 01:34:48 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
#Read data
mark2017 = spark.read.csv('./Datasets/2017.csv', header= True, inferSchema= True)
fullMark = spark.read.csv('./Datasets/full_exam.csv', header= True, inferSchema= True)
matinh = spark.read.csv('./Datasets/matinh.csv', header = True, inferSchema = True)

In [4]:
fullMark.printSchema()

root
 |-- SBD: integer (nullable = true)
 |-- Toan: double (nullable = true)
 |-- Van: double (nullable = true)
 |-- Ngoai ngu: double (nullable = true)
 |-- Ly: double (nullable = true)
 |-- Hoa: double (nullable = true)
 |-- Sinh: double (nullable = true)
 |-- Lich su: double (nullable = true)
 |-- Dia ly: double (nullable = true)
 |-- GDCD: double (nullable = true)
 |-- Year: integer (nullable = true)
 |-- MaTinh: integer (nullable = true)



In [5]:
fullMark.show()

+-------+----+----+---------+----+----+----+-------+------+----+----+------+
|    SBD|Toan| Van|Ngoai ngu|  Ly| Hoa|Sinh|Lich su|Dia ly|GDCD|Year|MaTinh|
+-------+----+----+---------+----+----+----+-------+------+----+----+------+
|8002816| 7.8|8.25|      6.2| 7.5|null| 7.0|   null|  null|null|2017|     8|
|8005707| 3.0|3.25|      3.2|2.75|null| 3.0|   null|  null|null|2017|     8|
|8004883| 6.0|5.25|      4.8|null|null|null|   null|  null|null|2017|     8|
|8004786| 2.4|3.75|     null|null|null|null|   null|  null|null|2017|     8|
|8004225| 4.2| 4.0|      3.0|null|null|null|   null|  null|null|2017|     8|
|8001882| 3.2|5.75|      3.8|null|null|null|   null|  null|null|2017|     8|
|8003558| 4.6| 6.0|     null|null|null|null|   null|  null|null|2017|     8|
|8004936| 4.0|5.75|      2.8|null|null|null|   null|  null|null|2017|     8|
|8001749| 1.4| 3.5|     null|null|null|null|   null|  null|null|2017|     8|
|8000321| 4.2|6.75|      4.0|4.75|null| 4.5|   null|  null|null|2017|     8|

In [6]:
# Remove all row which have 'year' = 2017 because of too many null value
fullMark = fullMark.filter(fullMark['year'] != 2017)

In [7]:
fullMark.show()

+--------+----+----+---------+----+----+----+-------+------+----+----+------+
|     SBD|Toan| Van|Ngoai ngu|  Ly| Hoa|Sinh|Lich su|Dia ly|GDCD|Year|MaTinh|
+--------+----+----+---------+----+----+----+-------+------+----+----+------+
|18010226| 3.0|3.75|      3.0|null|null|null|    3.0|   6.5|8.25|2018|    18|
|18010229| 8.8| 7.5|      9.0|null|null|null|    6.0|   9.0|8.25|2018|    18|
|18010232| 6.0| 5.5|      4.0|5.75| 5.5| 5.0|   null|  null|null|2018|    18|
|18010242| 3.4|5.75|      2.6|null|null|null|    3.5|  4.75|7.25|2018|    18|
|18010247| 3.8|6.75|      3.0|null|null|null|    3.5|  6.25| 8.0|2018|    18|
|18010252| 5.0| 6.5|      2.2| 2.0| 3.5|4.25|   null|  null|null|2018|    18|
|18010259| 2.4|6.25|      3.6|null|null|null|    5.0|   6.0| 8.0|2018|    18|
|18010262| 2.6|3.75|      3.0|null|null|null|    3.0|   4.5|7.75|2018|    18|
|18010265| 4.8|7.25|      2.2| 4.0| 5.0|4.25|   null|  null|null|2018|    18|
|18010268| 6.0| 6.5|      4.6|3.75|6.25|5.75|   null|  null|null

<h2> Map province and province code </h2?

In [4]:
import unidecode
def remove_accent(text):
    tmp = unidecode.unidecode(text)
    return tmp.lower().replace(' ', '')

In [5]:
from pyspark.sql.functions import udf

In [6]:
from pyspark.sql.types import StructField,StringType,IntegerType,StructType

In [7]:
remove_accent_udf = udf(remove_accent, StringType())
mark2017 = mark2017.withColumn('province', remove_accent_udf(mark2017['province']))

In [8]:
mark2017.printSchema()

root
 |-- student_id: integer (nullable = true)
 |-- province: string (nullable = true)
 |-- mathematics: double (nullable = true)
 |-- literature: double (nullable = true)
 |-- physics: double (nullable = true)
 |-- chemistry: double (nullable = true)
 |-- biology: double (nullable = true)
 |-- combined_natural_sciences: double (nullable = true)
 |-- history: double (nullable = true)
 |-- geography: double (nullable = true)
 |-- civic_education: double (nullable = true)
 |-- combined_social_sciences: double (nullable = true)
 |-- english: double (nullable = true)



In [13]:
# add 'year' columns with a default value 2017
mark2017 = mark2017.withColumn('year', lit(2017))

In [14]:
mark2017.limit(5).show()

+----------+---------+-----------+----------+-------+---------+-------+-------------------------+-------+---------+---------------+------------------------+-------+----+
|student_id| province|mathematics|literature|physics|chemistry|biology|combined_natural_sciences|history|geography|civic_education|combined_social_sciences|english|year|
+----------+---------+-----------+----------+-------+---------+-------+-------------------------+-------+---------+---------------+------------------------+-------+----+
|  47006585|binhthuan|        6.8|      5.25|   5.25|      5.5|    5.0|                     5.25|    4.0|      7.5|            8.5|                    6.67|    5.8|2017|
|  47003701|binhthuan|        3.4|      6.25|   null|     null|   null|                     null|   2.25|      3.5|           5.25|                    3.67|    3.6|2017|
|  47007840|binhthuan|        5.4|       5.5|   null|     null|   null|                     null|   7.25|     8.25|            8.5|                   

<h2> Remove all blank spaces and special characters in 'province' column

In [14]:
from pyspark.sql.functions import array_join, regexp_replace

In [15]:
matinh = matinh.withColumn("province", regexp_replace("province", "\\s+", ""))
matinh = matinh.withColumn('province', regexp_replace('province', "-", ""))

In [16]:
matinh.count()

63

In [17]:
matinh.limit(5).show()

+------+------------+
|MaTinh|    province|
+------+------------+
|     1|       hanoi|
|     2|tp.hochiminh|
|     3|    haiphong|
|     4|      danang|
|     5|     hagiang|
+------+------------+



In [18]:
matinh.printSchema()

root
 |-- MaTinh: integer (nullable = true)
 |-- province: string (nullable = true)



In [19]:
matinh.filter(matinh['MaTinh'] == 39).show()

+------+--------+
|MaTinh|province|
+------+--------+
|    39|  phuyen|
+------+--------+



<h2> Map province and province code respectively

In [20]:
mapedDF = mark2017.join(matinh, on=['province'], how = 'left')

In [21]:
mapedDF.printSchema()

root
 |-- province: string (nullable = true)
 |-- student_id: integer (nullable = true)
 |-- mathematics: double (nullable = true)
 |-- literature: double (nullable = true)
 |-- physics: double (nullable = true)
 |-- chemistry: double (nullable = true)
 |-- biology: double (nullable = true)
 |-- combined_natural_sciences: double (nullable = true)
 |-- history: double (nullable = true)
 |-- geography: double (nullable = true)
 |-- civic_education: double (nullable = true)
 |-- combined_social_sciences: double (nullable = true)
 |-- english: double (nullable = true)
 |-- year: integer (nullable = false)
 |-- MaTinh: integer (nullable = true)



In [22]:
mapedDF.createOrReplaceTempView("TAB")
spark.sql("select distinct province, MaTinh from TAB").count()

63

In [23]:
mapedDF =  mapedDF.withColumnRenamed('MaTinh', 'province_code')

<h1> Cleaned mark 2017 </h1>

In [24]:
from pyspark.sql.functions import isnan, isnull, when, count

In [25]:
mapedDF.printSchema()

root
 |-- province: string (nullable = true)
 |-- student_id: integer (nullable = true)
 |-- mathematics: double (nullable = true)
 |-- literature: double (nullable = true)
 |-- physics: double (nullable = true)
 |-- chemistry: double (nullable = true)
 |-- biology: double (nullable = true)
 |-- combined_natural_sciences: double (nullable = true)
 |-- history: double (nullable = true)
 |-- geography: double (nullable = true)
 |-- civic_education: double (nullable = true)
 |-- combined_social_sciences: double (nullable = true)
 |-- english: double (nullable = true)
 |-- year: integer (nullable = false)
 |-- province_code: integer (nullable = true)



In [26]:
checkingColumn = ['student_id', 'mathematics', 'literature', 'english', 'province', 'province_code', 'year', 'combined_natural_sciences', 'combined_social_sciences']

In [27]:
countNullDF = mapedDF.select([count(when(isnan(c) | isnull(c), c)).alias(c) for c in checkingColumn])

In [28]:
countNullDF.show()

+----------+-----------+----------+-------+--------+-------------+----+-------------------------+------------------------+
|student_id|mathematics|literature|english|province|province_code|year|combined_natural_sciences|combined_social_sciences|
+----------+-----------+----------+-------+--------+-------------+----+-------------------------+------------------------+
|         0|      17743|     37456| 120324|       0|            0|   0|                   468713|                  427120|
+----------+-----------+----------+-------+--------+-------------+----+-------------------------+------------------------+



In [64]:
mapedDF.count()

847348

In [36]:
mapedDF.select(col('province_code')).distinct().count()

63

In [32]:
cleanedMark_2017 = mapedDF.dropna(subset=["mathematics", "literature", "english"])

In [33]:
testDF = mapedDF.dropna(subset=["mathematics", "literature", "english"])

In [34]:
cleanedMark_2017.count()

721756

In [35]:
cleanedMark_2017.select(col('province_code')).distinct().count()

61

In [73]:
testDF.count()

721756

<h2> Remove all rows which has mark <= 1

In [101]:
cleanedMark_2017 = cleanedMark_2017.filter(~isnull(cleanedMark_2017.mathematics) & (cleanedMark_2017.mathematics > 1)) \
               .filter(~isnull(cleanedMark_2017.literature) & (cleanedMark_2017.literature > 1)) \
               .filter(isnull(cleanedMark_2017.physics) | (~isnull(cleanedMark_2017.physics) & (cleanedMark_2017.physics > 1))) \
               .filter(isnull(cleanedMark_2017.chemistry) | (~isnull(cleanedMark_2017.chemistry) & (cleanedMark_2017.chemistry > 1))) \
               .filter(isnull(cleanedMark_2017.biology) | (~isnull(cleanedMark_2017.biology) & (cleanedMark_2017.biology > 1))) \
               .filter(isnull(cleanedMark_2017.history) | (~isnull(cleanedMark_2017.history) & (cleanedMark_2017.history > 1))) \
               .filter(isnull(cleanedMark_2017.geography) | (~isnull(cleanedMark_2017.geography) & (cleanedMark_2017.geography > 1))) \
               .filter(isnull(cleanedMark_2017.civic_education) | (~isnull(cleanedMark_2017.civic_education) & (cleanedMark_2017.civic_education > 1))) \
               .filter(~isnull(cleanedMark_2017.english) & (cleanedMark_2017.english > 1))

In [102]:
cleanedMark_2017.count()

719488

<h1>Export mark_2017 to csv file

In [103]:
output = cleanedMark_2017.coalesce(1)
output.write.option('delimiter', ',').csv('./CleanedDatasets/Mark2017', header = True)

<h1> Reformat full mark, removed 2017 <h1>

In [104]:
fullMark.count()

4347281

In [105]:
fullMark.printSchema()

root
 |-- SBD: integer (nullable = true)
 |-- Toan: double (nullable = true)
 |-- Van: double (nullable = true)
 |-- Ngoai ngu: double (nullable = true)
 |-- Ly: double (nullable = true)
 |-- Hoa: double (nullable = true)
 |-- Sinh: double (nullable = true)
 |-- Lich su: double (nullable = true)
 |-- Dia ly: double (nullable = true)
 |-- GDCD: double (nullable = true)
 |-- Year: integer (nullable = true)
 |-- MaTinh: integer (nullable = true)



In [106]:
fullMark = fullMark.withColumnRenamed('SBD', 'student_id')\
                   .withColumnRenamed('Toan', 'mathematics')\
                   .withColumnRenamed('Van', 'literature')\
                    .withColumnRenamed('Ngoai ngu', 'english')\
                    .withColumnRenamed('Ly', 'physics')\
                    .withColumnRenamed('Hoa', 'chemistry')\
                    .withColumnRenamed('Sinh', 'biology')\
                    .withColumnRenamed('Lich su', 'history')\
                    .withColumnRenamed('Dia ly', 'geography')\
                    .withColumnRenamed('GDCD', 'civic_education')\
                    .withColumnRenamed('MaTinh', 'province_code')

In [107]:
mapedDF.printSchema()

root
 |-- province: string (nullable = true)
 |-- student_id: integer (nullable = true)
 |-- mathematics: double (nullable = true)
 |-- literature: double (nullable = true)
 |-- physics: double (nullable = true)
 |-- chemistry: double (nullable = true)
 |-- biology: double (nullable = true)
 |-- combined_natural_sciences: double (nullable = true)
 |-- history: double (nullable = true)
 |-- geography: double (nullable = true)
 |-- civic_education: double (nullable = true)
 |-- combined_social_sciences: double (nullable = true)
 |-- english: double (nullable = true)
 |-- year: integer (nullable = false)
 |-- province_code: integer (nullable = true)



In [108]:
fullMark.printSchema()

root
 |-- student_id: integer (nullable = true)
 |-- mathematics: double (nullable = true)
 |-- literature: double (nullable = true)
 |-- english: double (nullable = true)
 |-- physics: double (nullable = true)
 |-- chemistry: double (nullable = true)
 |-- biology: double (nullable = true)
 |-- history: double (nullable = true)
 |-- geography: double (nullable = true)
 |-- civic_education: double (nullable = true)
 |-- Year: integer (nullable = true)
 |-- province_code: integer (nullable = true)



In [109]:
from pyspark.sql.functions import avg, array
from pyspark.sql.types import DoubleType

In [110]:
avg_cols = udf(lambda array: sum(array)/len(array), DoubleType())

In [111]:
from pyspark.sql.functions import round

In [112]:
fullMark = fullMark.withColumn('combined_natural_sciences', round((fullMark.physics + fullMark.chemistry + fullMark.biology) / 3, 2))\
        .withColumn('combined_social_sciences', round((fullMark.history + fullMark.geography + fullMark.civic_education) /3 , 2))

In [113]:
fullMark.limit(5).show()

+----------+-----------+----------+-------+-------+---------+-------+-------+---------+---------------+----+-------------+-------------------------+------------------------+
|student_id|mathematics|literature|english|physics|chemistry|biology|history|geography|civic_education|Year|province_code|combined_natural_sciences|combined_social_sciences|
+----------+-----------+----------+-------+-------+---------+-------+-------+---------+---------------+----+-------------+-------------------------+------------------------+
|  18010226|        3.0|      3.75|    3.0|   null|     null|   null|    3.0|      6.5|           8.25|2018|           18|                     null|                    5.92|
|  18010229|        8.8|       7.5|    9.0|   null|     null|   null|    6.0|      9.0|           8.25|2018|           18|                     null|                    7.75|
|  18010232|        6.0|       5.5|    4.0|   5.75|      5.5|    5.0|   null|     null|           null|2018|           18|        

In [114]:
checkingColumn = ['student_id', 'mathematics', 'literature', 'english', 'province_code', 'year', 'combined_natural_sciences', 'combined_social_sciences']

In [115]:
countNullDF = fullMark.select([count(when(isnan(c) | isnull(c), c)).alias(c) for c in checkingColumn])

In [116]:
countNullDF.show()

+----------+-----------+----------+-------+-------------+----+-------------------------+------------------------+
|student_id|mathematics|literature|english|province_code|year|combined_natural_sciences|combined_social_sciences|
+----------+-----------+----------+-------+-------------+----+-------------------------+------------------------+
|         0|      33339|     68832| 503495|            0|   0|                  2818286|                 1970293|
+----------+-----------+----------+-------+-------------+----+-------------------------+------------------------+



In [117]:
fullMark.count()

4347281

In [118]:
cleanedMark_2018_2022 = fullMark.dropna(subset=["mathematics", "literature", "english"])

In [119]:
cleanedMark_2018_2022.count()

3826405

<h2> Remove all rows which has mark <= 1

In [120]:
cleanedMark_2018_2022 = cleanedMark_2018_2022.filter(~isnull(cleanedMark_2018_2022.mathematics) & (cleanedMark_2018_2022.mathematics > 1)) \
               .filter(~isnull(cleanedMark_2018_2022.literature) & (cleanedMark_2018_2022.literature > 1)) \
               .filter(isnull(cleanedMark_2018_2022.physics) | (~isnull(cleanedMark_2018_2022.physics) & (cleanedMark_2018_2022.physics > 1))) \
               .filter(isnull(cleanedMark_2018_2022.chemistry) | (~isnull(cleanedMark_2018_2022.chemistry) & (cleanedMark_2018_2022.chemistry > 1))) \
               .filter(isnull(cleanedMark_2018_2022.biology) | (~isnull(cleanedMark_2018_2022.biology) & (cleanedMark_2018_2022.biology > 1))) \
               .filter(isnull(cleanedMark_2018_2022.history) | (~isnull(cleanedMark_2018_2022.history) & (cleanedMark_2018_2022.history > 1))) \
               .filter(isnull(cleanedMark_2018_2022.geography) | (~isnull(cleanedMark_2018_2022.geography) & (cleanedMark_2018_2022.geography > 1))) \
               .filter(isnull(cleanedMark_2018_2022.civic_education) | (~isnull(cleanedMark_2018_2022.civic_education) & (cleanedMark_2018_2022.civic_education > 1))) \
               .filter(~isnull(cleanedMark_2018_2022.english) & (cleanedMark_2018_2022.english > 1))

In [121]:
cleanedMark_2018_2022.count()

3819805

<h1> Export to csv file

In [122]:
fullMarkOut = cleanedMark_2018_2022.coalesce(1)
fullMarkOut.write.option('delimiter', ',').csv('./CleanedDatasets/Mark_2018_2022', header = True)